<center><h2> Create employees JSON document </h2></center>

### Load libraries 

In [3]:
import json 
import uuid 
import random
import csv
import ipynb
import json
import pymongo as pym
import pickle
import pandas as pd 
import numpy as np
import ipynb.fs.full.custom_functions as cf
from datetime import datetime

In [4]:
#TO DO: ZABEZPIECZYC HASLO, USERNAME, ADRES DO BAZY etc.
client = pym.MongoClient(host = 'kti-lab1.ue.poznan.pl', port = 27017, username='ztbd47262', password='wd47262', authSource='ztbd47262', authMechanism='SCRAM-SHA-1')

In [5]:
db = client["ztbd47262"]

### Load data for generating employees

In [6]:
# load file with polish names. 
with open("data_for_JSON/names.csv", newline='', encoding='utf-8') as f:
    reader = csv.reader(f)
    names_list = list(filter(None, reader))
    del names_list[0]
    
#load data from GUS about polish surnames
data_surnames_men = pd.read_csv("data_for_JSON/surnames_gus_men.csv", skiprows=1, names = ["surname","number"], encoding='utf-8')
data_surnames_women = pd.read_csv("data_for_JSON/surnames_gus_women.csv", skiprows=1, names = ["surname","number"], encoding='utf-8')
#load data from BAEL about age of working ppl.
data_age_men = pd.read_excel("data_for_JSON/age_bael.xlsx", sheet_name = "men")
data_age_women = pd.read_excel("data_for_JSON/age_bael.xlsx", sheet_name = "women")

In [7]:
#Create column with perc value
data_surnames_men = cf.create_perc_col(data_surnames_men, ndigits = 10)
data_surnames_women = cf.create_perc_col(data_surnames_women, ndigits = 10)

In [8]:
data_age_men = cf.create_perc_col(data_age_men)
data_age_women = cf.create_perc_col(data_age_women)
data_age_men["age"] = data_age_men["age"].str.strip().str.replace("65  lat i więcej", "65-72")
data_age_women["age"] = data_age_women["age"].str.strip().str.replace("65  lat i więcej", "65-72")

### Generate values for employees JSON

In [9]:
number_of_emplo_in_center = 1000
emplo_list_dict = []

In [11]:
for i in range(number_of_emplo_in_center):
    emplo_dict = cf.generate_employee(names_list, data_surnames_men,data_age_men, data_surnames_women, data_age_women)
    emplo_list_dict.append(emplo_dict)

In [12]:
with open('created_tables_json/employees_pickle', 'wb') as fp:
    pickle.dump(emplo_list_dict, fp)

### Add to MongoDB

In [13]:
#db.create_collection("emplo_pro")

Collection(Database(MongoClient(host=['kti-lab1.ue.poznan.pl:27017'], document_class=dict, tz_aware=False, connect=True, authsource='ztbd47262', authmechanism='SCRAM-SHA-1'), 'ztbd47262'), 'emplo_pro')

In [14]:
collection_emplo_pro = db["emplo_pro"]

In [15]:
collection_emplo_pro.insert_many(emplo_list_dict)

In [16]:
client.close()